In [ ]:
import pandas as pd
import os
import logging


In [ ]:
from spatialdc.evaluation.evaluation import compute_num_rare

In [ ]:
from spatialdc.lda.model import train

In [19]:
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
PATH_TO_MODELS = '/mnt/home/vophuong/spatialDC/example/output/'
os.makedirs(PATH_TO_MODELS, exist_ok=True)

In [20]:
from spatialdc.utils.function import load_single_sample
PATH_TO_DATA = '/mnt/home/vophuong/spatialDC/data/'
corpus, pos = load_single_sample(PATH_TO_DATA)
sample_names = ['Sample_1']
from spatialdc.spatial.graph_construction import knn_graph_single_sample
n_neighbors = 4
knn_graph_matrix = knn_graph_single_sample(pos, n_neighbors, sample_name = 'Sample_1')

In [21]:
n_topics = 12

In [22]:
spatialdc_model = train(
                corpus=corpus,
                graph_matrices= knn_graph_matrix,
                nu_penalty= 10,
                n_topics=n_topics
                )  

In [ ]:
perplexities = []
num_rares = []

path_to_model = os.path.join(PATH_TO_MODELS, f'spatialdc_topics={n_topics}_knn={n_neighbors}.pkl')
path_to_gamma = os.path.join(PATH_TO_MODELS, f'gamma_spatialdc_topics={n_topics}_knn={n_neighbors}.csv')
path_to_beta = os.path.join(PATH_TO_MODELS, f'beta_spatialdc_topics={n_topics}_knn={n_neighbors}.csv')
path_to_ppxt = os.path.join(PATH_TO_MODELS, f'ppxt_spatialdc_topics={n_topics}_knn={n_neighbors}.csv')


# Extract the beta matrix (topic-word distributions)
if hasattr(spatialdc_model, 'components_'):
    beta_matrix = spatialdc_model.components_
else:
    raise AttributeError("The loaded model does not have 'components_' attribute.")

# Normalize the beta matrix
# beta_matrix = beta_matrix / beta_matrix.sum(axis=1, keepdims=True)

# Convert to DataFrame
beta_df = pd.DataFrame(beta_matrix)

# Save to CSV
beta_df.to_csv(path_to_beta, index=False)

# Save gamma
gamma = spatialdc_model.topic_weights
gamma_df = pd.DataFrame(gamma)
gamma_df.to_csv(path_to_gamma, index=True)

# Compute metrics
perplexity = spatialdc_model.perplexity(corpus)
num_rare = compute_num_rare(spatialdc_model, corpus, 0.05)

perplexities.append(perplexity)
num_rares.append(num_rare)

results_df = pd.DataFrame({
    'n_topics': n_topics,
    'Perplexity': perplexities,
    'NumRare': num_rares
})
results_df.to_csv(path_to_ppxt, index=True)
logging.info(f"Results saved to {path_to_ppxt}")

2025-07-25 12:34:52,587 - INFO - Results saved to /mnt/home/vophuong/spatialDC/example/output/ppxt_spatialdc_topics=12_knn=4.csv
